You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Artificial Intelligence Blog Article

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and news in AI

Key Points:
1. Latest Trends in AI:
- Advancements in machine learning algorithms
- Increased adoption of AI in healthcare, finance, and retail sectors
- Growth of AI-powered virtual assistants and chatbots

2. Key Players in AI:
- Google's DeepMind
- IBM Watson
- Amazon Web Services
- Micro

I now can give a great answer

Final Answer: 

# Unveiling the Future: A Look into Artificial Intelligence Trends and Innovations

In a world driven by technological advancements, Artificial Intelligence (AI) stands at the forefront of innovation, revolutionizing various industries with its capabilities. From healthcare to finance and retail sectors, AI has become an integral part of modern-day operations, enhancing efficiency and productivity. As we delve deeper into the realm of AI, it becomes increasingly crucial to stay updated on the latest trends and news shaping this rapidly evolving landscape.

## Latest Trends in AI: Embracing the Power of Machine Learning

One of the key trends in AI revolves around the advancements in machine learning algorithms, enabling machines to learn from data and improve their performance over time. This has led to increased adoption of AI in diverse sectors, with businesses leveraging AI-powered solutions to gain a competitive edge. Moreover, the ris

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Future: A Look into Artificial Intelligence Trends and Innovations

In a world driven by technological advancements, Artificial Intelligence (AI) stands at the forefront of innovation, revolutionizing various industries with its capabilities. From healthcare to finance and retail sectors, AI has become an integral part of modern-day operations, enhancing efficiency and productivity. As we delve deeper into the realm of AI, it becomes increasingly crucial to stay updated on the latest trends and news shaping this rapidly evolving landscape.

## Latest Trends in AI: Embracing the Power of Machine Learning

One of the key trends in AI revolves around the advancements in machine learning algorithms, enabling machines to learn from data and improve their performance over time. This has led to increased adoption of AI in diverse sectors, with businesses leveraging AI-powered solutions to gain a competitive edge. Moreover, the rise of AI-powered virtual assistants and chatbots has transformed customer interactions, providing personalized experiences and streamlining communication processes. As AI continues to evolve, we can expect to witness further innovations that redefine the way we work and interact with technology.

## Key Players in AI: Pioneering the Future of Artificial Intelligence

In the realm of AI, several key players have emerged as leaders in driving innovation and pushing the boundaries of what is possible. Companies such as Google's DeepMind, IBM Watson, Amazon Web Services, and Microsoft Azure are at the forefront of AI research and development, pushing the limits of what AI can achieve. Their groundbreaking work in AI applications has not only transformed industries but also paved the way for future advancements in the field. By collaborating with these industry giants, businesses can harness the power of AI to unlock new opportunities and drive growth in the digital age.

## Noteworthy News in AI: Exploring Ethical Considerations and Climate Change Mitigation

In addition to technological advancements, AI has also sparked discussions around ethical considerations in its development and deployment. Breakthroughs in natural language processing have raised questions about data privacy and algorithmic bias, prompting the need for ethical guidelines to govern AI usage. Furthermore, AI applications in climate change mitigation have showcased the potential of AI to address global challenges and drive sustainable solutions. By staying informed on these noteworthy developments, we can actively participate in shaping the future of AI and ensuring its responsible use for the greater good.

In conclusion, Artificial Intelligence continues to shape the future of technology and innovation, with its vast potential to revolutionize industries and drive growth. By staying abreast of the latest trends, key players, and noteworthy news in AI, we can harness the power of AI to create a more sustainable and technologically advanced world. As we navigate the complexities of AI development, it is essential to prioritize ethical considerations and leverage AI for positive impact. Let us embrace the future of AI with curiosity and responsibility, paving the way for a brighter tomorrow.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Blog Article on Social Media Marketing Trends in 2021

Introduction:
- Brief overview of the importance of staying updated on social media marketing trends
- Mention of the rapid evolution of social media platforms and strategies

Key Points:
1. Latest Trends:
- Rise of short-form video content on platforms like TikTok and Instagram Reels
- Increased focus on influencer marketing and user-generated content
- Growth of social commerce and shoppable posts on platforms like Facebook and Instagram

2. Key Pl

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).